In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pingouin as pg
import researchpy as rp
import scipy.stats as stats

In [ ]:
# add OASIS dataset path here...
OASIS_DATASET_PATH = 'OASIS/subset01'
DATASET_PATH = 'dataset.csv'

In [13]:
dataset = pd.read_csv(DATASET_PATH, index_col='Index')
dataset.head()

,Unnamed: 0,Participant id,Image order,Image name,HSP score,HSP class,Income,Valence,Arousal,Time,...,Gender code,Age,Ideology,Ideology code,Race,Race code,Nationaly,Nationaly code,Education,Education code
Index,,,,,,,,,,,,,,,,,,,,,
0,0,139,1,Acorns 2,79,1,0,4,1,326.0,...,1,21,Neutral,4,White,1,Azari,2,Bachelors,3
1,1,139,2,Cold 2,79,1,0,5,5,83.5,...,1,21,Neutral,4,White,1,Azari,2,Bachelors,3
2,2,139,3,Hang gliding 2,79,1,0,6,6,54.5,...,1,21,Neutral,4,White,1,Azari,2,Bachelors,3
3,3,139,4,Alcohol 2,79,1,0,1,1,154.0,...,1,21,Neutral,4,White,1,Azari,2,Bachelors,3
4,4,139,5,Collaboration 1,79,1,0,4,1,57.5,...,1,21,Neutral,4,White,1,Azari,2,Bachelors,3


In [3]:
import keras
from keras import layers, models, losses, metrics, optimizers
from functools import lru_cache
import cv2


@lru_cache(maxsize=500)
def load_img(name, size):
    img = cv2.imread(OASIS_DATASET_PATH + '/' + name + '.jpg')
    img = cv2.resize(img, size[::-1])
    return img / 255


class DataGenerator():
    def __init__(self, df, batch_size, dataset_split, img_size):
        indices = list(range(len(df)))
        # np.random.shuffle(indices)

        train_idx_end = int(dataset_split[0] * len(indices))
        val_idx_end = int(dataset_split[1] * len(indices))

        self.indices = indices
        self.batch_size = batch_size
        self.train_set = indices[:train_idx_end]
        self.validation_set = indices[train_idx_end:val_idx_end]
        self.test_set = indices[val_idx_end:]
        self.img_size = img_size
        self.df = df

    def train_split(self):
        return DataGenerator.DataGeneratorObj(self.train_set, self.batch_size, self.img_size, self.df)

    def validation_split(self):
        return DataGenerator.DataGeneratorObj(self.validation_set, self.batch_size, self.img_size, self.df)

    def test_split(self):
        return DataGenerator.DataGeneratorObj(self.test_set, self.batch_size, self.img_size, self.df)


    class DataGeneratorObj(keras.utils.Sequence):
        def __init__(self, indices, batch_size, img_size, df):
            self.indices = indices
            self.batch_size = batch_size
            self.img_size = img_size
            self.df = df

        def __len__(self):
            return int(len(self.indices) / self.batch_size)

        def __getitem__(self, index):

            batch = self.indices[index * self.batch_size: (index + 1) * self.batch_size]
            x1, x2, y = [], [], []

            for batch_index in batch:
                row = self.df.iloc[batch_index]
                img = load_img(row['Image name'], self.img_size)
                selections = [*map(int, row['Selection'].split(';'))]
                selection_intervals = [*map(float, row['Selection interval'].split(';'))]

                padded_selections = np.pad(selections, (0, 9 - len(selections)))
                padded_selection_intervals = np.pad(selection_intervals, (0, 9 - len(selection_intervals)))

                features = [
                    row['Age'],
                    row['Ideology code'],
                    row['Race code'],
                    row['Nationaly code'],
                    row['Education code'],
                    row['Gender code'],
                    row['Income'],
                    row['Valence'],
                    row['Arousal'],
                    *padded_selections,
                    *padded_selection_intervals
                ]

                x1.append(img)
                x2.append(features)
                y.append(row['HSP score'])

            x1 = np.array(x1)
            x2 = np.array(x2)
            y = np.array(y)

            return (x1, x2), y

In [4]:
from net_vit_model import create_vit_classifier

def make_model(input_shape, output_shape):
    image_size = input_shape[0][0]
    patch_size = 6
    num_patches = (image_size // patch_size) ** 2
    projection_dim = 64
    num_heads = 4
    transformer_units = [projection_dim * 2, projection_dim]
    transformer_layers = 8
    mlp_head_units = [2_048, 1_024]
    vit = create_vit_classifier(
        input_shape[0],
        patch_size,
        num_patches,
        projection_dim,
        transformer_layers,
        num_heads,
        transformer_units,
        mlp_head_units,
        1_024)

    input2 = layers.Input(shape=(input_shape[1],))
    x = layers.Dense(64, activation="relu", name=f'm_dense_1')(input2)
    x = layers.Dense(32, activation="relu", name=f'm_dense_2')(x)
    x = layers.Dense(8, activation="relu", name=f'm_dense_3')(x)
    x = models.Model(inputs=input2, outputs=x)

    combined = layers.concatenate([x.output, vit.output])

    z = layers.Dense(1024, activation="relu", name=f'm_dense_4')(combined)
    z = layers.Dense(256, activation="relu", name=f'm_dense_5')(z)
    z = layers.Dense(64, activation="relu", name=f'm_dense_6')(z)
    z = layers.Dense(output_shape, name=f'm_dense_7')(z)

    model = models.Model(inputs=[vit.input, x.input], outputs=z)
    return model

In [5]:
import keras.backend as backend
import tensorflow as tf
def threshold_accuracy(thresh):
    def t_accuracy(_true, _pred):
        diff = backend.abs((backend.abs(_pred)) - _true)
        diff = tf.where(diff <= thresh, 1, 0)
        return backend.sum(diff) / tf.size(diff, out_type=tf.dtypes.int32)
    
    return t_accuracy

In [8]:
img_size = (500, 500, 3)
gen = DataGenerator(dataset, 16, [0.7, 0.9], img_size[:-1])
train_split = gen.train_split()
validation_split = gen.validation_split()
test_split = gen.test_split()

In [9]:
model = make_model([img_size, 27], 1)
model.compile(optimizer=optimizers.adam_v2.Adam(),
                loss=losses.mse,
                metrics=[threshold_accuracy(5)])

In [ ]:
model.fit(train_split, validation_data=validation_split, epochs=1_000)